# IGRINS CA1 geometry function

Here we make a function that takes in grating properties and computes its properties.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format='retina'

### Fixed physical properties

In [ ]:
sigma = 27.36 # micron

In [ ]:
groove_top_width = 9.95 # micron

Here is a cartoon of the IGRINS CA-1 grating from [this slideshow](https://speakerdeck.com/gully/ideas-for-metrology-of-silicon-diffractive-optics?slide=45).

In [ ]:
%%html
<img src="../IGRINS_CA1_schematic.png" width=300></img>

The angle $a$ is the fixed $70.53^\circ$ from the Si crystal planes, $b$ is $\arctan{3}$, and c is whatever is left over to add up to 180.

In [ ]:
a_degrees = 70.53 # "Fixed" Si crystal plane angle

In [ ]:
b_degrees = np.degrees(np.arctan(3)) # Blaze angle

## Create the function

In [ ]:
def grating_geometry(sigma, groove_top_width, b_degrees, a_degrees):
    """Compute the grating groove geometry from math
    
    Parameters:
    ----------
    
    sigma (micron): 
        grating groove spacing or "groove constant"
    groove_top_width (micron):
        Si grating groove top width defined by UV photomask chrome stripes
    b_degrees (degrees):
        The b angle from Figure 1 above, often the blaze angle
    a_degrees (degrees):
        The a angle from Figure 1 above, also called the Si apex angle
    """
    
    c_degrees = 180 - a_degrees - b_degrees # Complement must add up to 180
    
    a_radians, b_radians, c_radians = (np.radians(angle) for angle in 
                                   [a_degrees, b_degrees, c_degrees])
    
    # Compute the line segments A, B, C with the Law of Sines:
    A_micron = sigma - groove_top_width
    
    fixed_ratio = np.sin(a_radians) / A_micron
    B_micron = np.sin(b_radians) / fixed_ratio
    C_micron = np.sin(c_radians) / fixed_ratio
    
    # Compute the vertices, define vertex `c` as the origin
    c_vertex = np.array([0.0, 0.0])
    b_vertex = c_vertex + np.array([B_micron, 0.0])
    a_vertex = np.array([B_micron*np.cos(c_radians), -B_micron*np.sin(c_radians)])
    
    # Define one other key point, the coordinate of the start of the groove top.
    d_vertex = np.array([-groove_top_width, 0.0])
    
    return {'angle_degrees':(a_degrees, b_degrees, c_degrees),
            'angle_radians':(a_radians, b_radians, c_radians),
            'segments_micron':(A_micron, B_micron, C_micron),
           'vertices_micron':(a_vertex, b_vertex, c_vertex, d_vertex)}

In [ ]:
geometry_dict = grating_geometry(sigma, groove_top_width, b_degrees, a_degrees)

In [ ]:
geometry_dict

## Add a bokeh interactive dashboard

## Add incident rays

In [ ]:
incidence_angle_degrees = -44.6
incidence_angle_radians = np.radians(incidence_angle_degrees)

Determine the coordinates of the beam.  The rays come in from infinity, but let's draw them has having a fixed length for illustration purposes.

In [ ]:
far = 100.0

In [ ]:
ray1_vertex = np.array([-far*np.sin(incidence_angle_radians), 
                                far*np.cos(incidence_angle_radians)])

Determine the coordinate at which the marginal ray is shadowed:

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot([c_vertex[0], a_vertex[0]], [c_vertex[1], a_vertex[1]], lw=5, label='B')
plt.plot([a_vertex[0], b_vertex[0]], [a_vertex[1], b_vertex[1]], lw=5, label='C')
plt.plot([c_vertex[0], d_vertex[0]], [c_vertex[1], d_vertex[1]], lw=5, label='Groove Top')

# Plot the incoming rays
plt.plot([c_vertex[0], ray1_vertex[0]], [c_vertex[1], ray1_vertex[1]], 
         color='#ff4600', lw=2, label='Incoming Rays')
plt.plot([c_vertex[0]+B_micron, ray1_vertex[0]+B_micron], [c_vertex[1], ray1_vertex[1]], 
         color='#ff4600', lw=2)
plt.plot([c_vertex[0]+B_micron, -ray1_vertex[0]+B_micron], [c_vertex[1], -ray1_vertex[1]], 
         color='#ff4600', lw=2, linestyle='dotted', label='Marginal Ray')

plt.text(B_micron/2, 1, 'A')
plt.text(b_vertex[0]-2, b_vertex[1]-2, r'$\beta$')
plt.text(2,-1, r'$\gamma$')
plt.text(17,-5, 'h')
plt.text(2,-4, 'w')
plt.text(+7,-4, r'$\phi$')
plt.text(+14,-1, r'$\omega$')


plt.plot([0.0, 0.0], [0.0, 15], linewidth=2, color='k', marker='^', markersize=10)

plt.axhline(0.0, linestyle='dashed', color='k', alpha=0.5)

plt.xlim(-20, 30)
plt.ylim(-20, 30)
plt.xlabel('$x$ ($\mu$m)');
plt.ylabel('$y$ ($\mu$m)')
plt.legend();

We have $\gamma$, and $\beta$, so we can get $\alpha$:

In [ ]:
gamma_degrees = c_degrees

In [ ]:
omega_degrees = 90 - np.abs(incidence_angle_degrees)
omega_radians = np.radians(omega_degrees)

In [ ]:
phi_degrees = 180 - omega_degrees - gamma_degrees
phi_radians = np.radians(phi_degrees)

We want to find the length of the linesegment.  Let's use angle-side-angle `ASA`:

$$\frac{\sin{\phi}}{A} \,=\, \frac{\sin{\omega}}{w} \,=\, \frac{\sin{\gamma}}{\mathrm{undef}}$$

solve for $w$:

$$w \,=\, A \frac{\sin{\omega}}{\sin{\phi}}$$


In [ ]:
w_micron = A_micron * np.sin(omega_radians)/np.sin(phi_radians)

In [ ]:
w_micron

Yay! We did it!